# JuliaでのLoggingのやり方
参考文献：https://docs.julialang.org/en/v1/stdlib/Logging/#Logging

## @warnで警告メッセージを表示

In [1]:
@warn "Abandon printf debugging, all ye who enter here!"

┌ Warning: Abandon printf debugging, all ye who enter here!
└ @ Main In[1]:1


## @debug

In [2]:
@debug "The sum of some values $((sum(rand(100))))"

@debugはデフォルトだと何も出力しない．loggingをオンにすれば出力されるし，オフにすれば出力されない．楽ちんだ．

## loggingツールを使うとkeyとvalueのペアのセットに対して，データを繋げることができる
例えば，以下の例．

In [3]:
A = ones(Int, 4, 4)
v = ones(100)
@info "Some variables" A s=sum(v)
#^key ^values 

┌ Info: Some variables
│   A = [1 1 1 1; 1 1 1 1; 1 1 1 1; 1 1 1 1]
│   s = 100.0
└ @ Main In[3]:3


@infoによって，"Some variables"という文字列が二つの変数Aとsを出力することができる．
これで変数の値やプログラムの状態を簡単に確認できる．
全てのloggingマクロ`@debug`, `@info`, `@warn`, `@errro`は共通の機能を持っている．いかにヘルプを記載する：

In [4]:
?@info

```
@debug message  [key=value | value ...]
@info  message  [key=value | value ...]
@warn  message  [key=value | value ...]
@error message  [key=value | value ...]

@logmsg level message [key=value | value ...]
```

Create a log record with an informational `message`.  For convenience, four logging macros `@debug`, `@info`, `@warn` and `@error` are defined which log at the standard severity levels `Debug`, `Info`, `Warn` and `Error`.  `@logmsg` allows `level` to be set programmatically to any `LogLevel` or custom log level types.

`message` should be an expression which evaluates to a string which is a human readable description of the log event.  By convention, this string will be formatted as markdown when presented.

The optional list of `key=value` pairs supports arbitrary user defined metadata which will be passed through to the logging backend as part of the log record.  If only a `value` expression is supplied, a key representing the expression will be generated using [`Symbol`](@ref). For example, `x` becomes `x=x`, and `foo(10)` becomes `Symbol("foo(10)")=foo(10)`.  For splatting a list of key value pairs, use the normal splatting syntax, `@info "blah" kws...`.

There are some keys which allow automatically generated log data to be overridden:

  * `_module=mod` can be used to specify a different originating module from the source location of the message.
  * `_group=symbol` can be used to override the message group (this is normally derived from the base name of the source file).
  * `_id=symbol` can be used to override the automatically generated unique message identifier.  This is useful if you need to very closely associate messages generated on different source lines.
  * `_file=string` and `_line=integer` can be used to override the apparent source location of a log message.

There's also some key value pairs which have conventional meaning:

  * `maxlog=integer` should be used as a hint to the backend that the message should be displayed no more than `maxlog` times.
  * `exception=ex` should be used to transport an exception with a log message, often used with `@error`. An associated backtrace `bt` may be attached using the tuple `exception=(ex,bt)`.

# Examples

```julia
@debug "Verbose debugging information.  Invisible by default"
@info  "An informational message"
@warn  "Something was odd.  You should pay attention"
@error "A non fatal error occurred"

x = 10
@info "Some variables attached to the message" x a=42.0

@debug begin
    sA = sum(A)
    "sum(A) = $sA is an expensive operation, evaluated only when `shouldlog` returns true"
end

for i=1:10000
    @info "With the default backend, you will only see (i = $i) ten times"  maxlog=10
    @debug "Algorithm1" i progress=i/10000
end
```


## ログをファイルに書き込む

In [5]:
using Logging
io = open("log.txt", "w+")
logger = SimpleLogger(io)
with_logger(logger) do 
    @info "a context specific log message"
end
flush(io)
global_logger(logger)
@info "a global log message"
close(io)